<div style='border-bottom: 1px solid #444; height: 50px'>
    <h1 style='text-align: center; color: #444; font-variant: small-caps; font-weight: 700; font-family: Bell Gothic Std, Verdana; font-size: 50px;'> 
    CIFAR Net
    </h1>
</div>

In [1]:
from sys import path
path.insert(0, '/home/clement/Documents/Maitrise/Code/src/lib/')
from utilities import log, Process
log

<IPython.core.display.Javascript object>

VBox(children=(HBox(children=(HSpace(value='', layout=Layout(width='400px')), LogToolBar(children=(ToolButton(button_color='#bdbdbd', html='<i class="fa fa-info-circle" style="font-size: 20px; color: #2176ff;"></i>', layout=Layout(height='40px', width='40px'), pressed_color='#eee', toggleable=True, value=True), ToolButton(button_color='#bdbdbd', html='<i class="fa fa-exclamation-triangle" style="font-size: 20px; color: #eba427;"></i>', layout=Layout(height='40px', width='40px'), pressed_color='#eee', toggleable=True, value=True), ToolButton(button_color='#bdbdbd', html='<i class="fa fa-exclamation-circle" style="font-size: 20px; color: #c71e1e;"></i>', layout=Layout(height='40px', width='40px'), pressed_color='#eee', toggleable=True, value=True), ToolButton(button_color='#bdbdbd', html='<i class="fa fa-code" style="font-size: 20px; color: #111;"></i>', layout=Layout(height='40px', width='40px'), pressed_color='#eee', toggleable=True, value=True), HTML(value='', layout=Layout(width='25px')), Text(value='', layout=Layout(width='200px'), placeholder='log filters'), HTMLButton(button_color='#bdbdbd', html='Aa', layout=Layout(height='40px', width='40px'), pressed_color='#eee', toggleable=True, value=True)))), layout=Layout(align_items='flex-end', justify_content='space-between')), Accordion(children=(VBox(children=(Accordion(layout=Layout(max_height='500px', width='100%')), VSpace(value='', layout=Layout(height='25px')), LogView(layout=Layout(height='200px', overflow_y='auto'))), layout=Layout(overflow_y='auto')),), _titles={'0': 'global'})))

## Import

In [2]:
import numpy as np


In [3]:
from IO_tools.dataset import cifar10_datasets
from NNet.Layers import *
from flow.utils import Concat, CropConcat
from NNet import MultipleIOClassifier
from NNet.engine import TrainEngine
from NNet import load_znet
from NNet.optimizer import *
from NNet.engine import Context
from NNet.engine.context import Timestamp
import pickle

Using cuDNN version 7103 on context None
Mapped name None to device cuda: GeForce GTX 860M (0000:01:00.0)


## Constante

# Initialisation

In [4]:
ini_process = Process('Initialisation', start=True)

## Initialisation Dataset

In [5]:
d = cifar10_datasets()
d_train = d['train'].split_sets(train=0.8, valid=0.2)
datasets = {'train': d_train['train'], 'validation': d_train['valid'], 'test': d['test']}
train = datasets['train']

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/clement/miniconda3/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-6d79ed142ef0>", line 1, in <module>
    d = cifar10_datasets()
  File "/home/clement/Documents/Maitrise/Code/src/lib/IO_tools/dataset/std_datasets.py", line 66, in cifar10_datasets
    with tar.extractfile(prefix+'data_batch_%i' % (i+1)) as f_data:
  File "/home/clement/miniconda3/envs/py35/lib/python3.5/tarfile.py", line 2064, in extractfile
    tarinfo = self.getmember(member)
  File "/home/clement/miniconda3/envs/py35/lib/python3.5/tarfile.py", line 1738, in getmember
    tarinfo = self._getmember(name)
  File "/home/clement/miniconda3/envs/py35/lib/python3.5/tarfile.py", line 2319, in _getmember
    members = self.getmembers()
  File "/home/clement/miniconda3/envs/py35/lib/python3.5/tarfile.py", line 1749, in getmembers
    self._load()       

EOFError: Compressed file ended before the end-of-stream marker was reached

## Définition Réseau

In [ ]:
with Process('Réseau'):
    net = MultipleIOClassifier()
    
    (Input(net, 'x') << ConvLayer(64, 5, activation='relu') 
                     << MaxPoolLayer() << LRN(k=1., alpha=1e-3/9., beta=0.75)
                     << ConvLayer(64, 5, activation='relu')
                     << LRN(k=1., alpha=1e-3/9., beta=0.75) << MaxPoolLayer()
                     << ConvLayer(64, 3, BN=True, activation='relu') << MaxPoolLayer()
                     << FullyConnected(384,  activation='relu')
                     << FullyConnected(192,  activation='relu')
                     << FullyConnected(10,  activation=None)
                     << ClassificationOutput('y', nb_class=10, label_datatype=DataType.SCALAR, group_by='notGrouped'))

    net.end_net()
    
    net.build_model_from_dataset(train)

## Fin d'initialisation

In [ ]:
ini_process.succeed()

#  Apprentissage

In [ ]:
e = TrainEngine(net, datasets, batch_size=64, optimizer=Adam(),
                macro_batch_size=100)

In [ ]:
c = Context()
e.train(10, c=c, test_period=500)